In [15]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import openpyxl
from ipywidgets import widgets, HBox
from IPython.display import display
from IPython.display import HTML

# Function to read an excel file and store it as a pandas DataFrame
def read_excel_file(filepath):
    # Load the excel file
    wb = openpyxl.load_workbook(filepath)
    # Select the first sheet
    sheet = wb.active
    # Store the sheet data in a pandas DataFrame
    data = pd.DataFrame(sheet.values)
    # Extract the table headers and store them in a list
    headers = list(data.iloc[0])
    # Remove the first row (containing the headers)
    data = data.iloc[1:]
    # Set the column names to the extracted headers
    data.columns = headers
    return data

# Function to plot the selected data
def plot_data(data, x_col, y_col):
    # Create a scatter plot using plotly
    fig = px.scatter(data, x=x_col, y=y_col)
    # plot using plotly offline in a separate window
    pyo.plot(fig,filename='output.html', auto_open=True)
    #pyo.iplot(fig)
    #fig.show()
    #Hyperlink the generated plot
    return HTML(f'<a href="output.html" target="_blank">Open Plot</a>')

# Create a button to select an excel file
upload_button = widgets.FileUpload(
    accept='.xlsx',  # Only allow xlsx files
    multiple=False  # Only allow one file to be uploaded
)

# Create a multi-select dropdown list with the table headers
headers = []  # placeholder list for headers
columns_dropdown = widgets.SelectMultiple(
    options=headers,
    description='Select Columns:',
    disabled=False
)

# Create a button to plot the selected data
plot_button = widgets.Button(description="Plot")

# Align the plot button and the upload button on the same line
button_layout = HBox([upload_button,columns_dropdown, plot_button])
display(button_layout)

def on_upload(change):
    # Get the uploaded file's data and store it in a DataFrame
    file = change['new'][0]
    data = read_excel_file(file.name)
    # Get the headers from the DataFrame
    headers.extend(data.columns)
    columns_dropdown.options=headers
    columns_dropdown.value=headers[:2]
    # Create a multi-select dropdown list with the table headers
    #display(columns_dropdown)
    # Create a button to plot the selected data
    #display(plot_button)
    def on_plot_clicked(b):
        # Extract the selected columns
        x_col, y_col = columns_dropdown.value
        # Plot the selected data
        link = plot_data(data, x_col, y_col)
        display(link)
    plot_button.on_click(on_plot_clicked)
    
upload_button.observe(on_upload, 'value')